In [6]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
import numpy as np

In [9]:
sonnets = []
with open("../data/shakespeare.txt") as f:
    line = f.readline()
    while line:
        # Flag start of sonnet, read in next 14 lines
        if any(char.isdigit() for char in line):
            curr_sonnet = ""
            for i in range(14):
                curr_sonnet += f.readline().strip().lower()
                curr_sonnet += "\n" if i != 13 else ""
            sonnets.append(curr_sonnet)
        line = f.readline()
        
# Vectorization prep
chars = sorted(list(set("".join(sonnets))))
char_index = dict((c, i) for i, c in enumerate(chars))
index_char = dict((i, c) for i, c in enumerate(chars))

# Read subsequences from each sonnet, add to training list
# Don't read across sonnets?
length = 40
step = 1
tr_data = []
tar_char = []
for s in sonnets:
    for i in range(0, len(s) - length, step):
        tr_data.append(s[i:i+length])
        tar_char.append(s[i+length])

# Vectorize training data
X = np.zeros((len(tr_data), length, len(chars)), dtype=np.bool)
Y = np.zeros((len(tr_data), len(chars)), dtype=np.bool)

for i, seq in enumerate(tr_data):
    for j, char in enumerate(seq):
        X[i, j, char_index[char]] = 1
    Y[i, char_index[tar_char[i]]] = 1

In [10]:
model = Sequential()
model.add(LSTM(128, input_shape=(length, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               85504     
_________________________________________________________________
dense_1 (Dense)              (None, 38)                4902      
Total params: 90,406
Trainable params: 90,406
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X, Y, epochs=1)

Epoch 1/1
80800/87316 [==========================>...] - ETA: 4s - loss: 2.2599